In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama


models_folder = "./models"
model_name = "TheBloke/Llama-2-13B-Chat-GGML"
file_name = "llama-2-13b-chat.ggmlv3.q4_K_M.bin"

file_path = hf_hub_download(repo_id=model_name, filename=file_name, local_dir=models_folder)
model = Llama(file_path, n_ctx=4096)


In [ ]:
history = [[""], ""]
history[0][0] = "hey! how are you?"

kwargs = dict(temperature=0.1, top_p=0.9)
kwargs["max_tokens"] = 128

instruction = "Hey! How are you?"
output = ""

def get_output(instruction):
    output = ""
    for chunk in model(prompt=instruction, stream=True, **kwargs):
        token = chunk["choices"][0]["text"]
        output += token

    print(output)
    return output

output = get_output(instruction)
output

In [27]:
text = """Quantum mechanics is a fundamental theory in physics that provides a description of the physical properties of nature at the scale of atoms and subatomic particles.[2]: 1.1  It is the foundation of all quantum physics including quantum chemistry, quantum field theory, quantum technology, and quantum information science.

Classical physics, the collection of theories that existed before the advent of quantum mechanics, describes many aspects of nature at an ordinary (macroscopic) scale, but is not sufficient for describing them at small (atomic and subatomic) scales. Most theories in classical physics can be derived from quantum mechanics as an approximation valid at large (macroscopic) scale.[3]

Quantum mechanics differs from classical physics in that energy, momentum, angular momentum, and other quantities of a bound system are restricted to discrete values (quantization); objects have characteristics of both particles and waves (wave–particle duality); and there are limits to how accurately the value of a physical quantity can be predicted prior to its measurement, given a complete set of initial conditions (the uncertainty principle).

Quantum mechanics arose gradually from theories to explain observations that could not be reconciled with classical physics, such as Max Planck's solution in 1900 to the black-body radiation problem, and the correspondence between energy and frequency in Albert Einstein's 1905 paper, which explained the photoelectric effect. These early attempts to understand microscopic phenomena, now known as the "old quantum theory", led to the full development of quantum mechanics in the mid-1920s by Niels Bohr, Erwin Schrödinger, Werner Heisenberg, Max Born, Paul Dirac and others. The modern theory is formulated in various specially developed mathematical formalisms. In one of them, a mathematical entity called the wave function provides information, in the form of probability amplitudes, about what measurements of a particle's energy, momentum, and other physical properties may yield."""


prompt = f"You are a GOD-like AGI that is a top level expert in all of the fields in the world. "  + f"Your task is to design a a question-answer pair on the given text below. "  + f"The person reading this is very smart, so don't go easy on them. "  + f"Make sure that the exam question is relevant to the given text."  + f"Think step-by-step, and make sure that the answer is not too obvious. "  + f"Don't include possible answers that are like 'any of the above' or 'none of the above'. "  + f"Format the answer as follows:\n " + f"Question: (write your question here)\n" + f"Answer: (write your answer here)\n" + f". Start your response with 'Question: ' . Text that you will use to generate question-answer pair: {text}"
    

prompt = prompt.replace("\"", "")

prompt

"You are a GOD-like AGI that is a top level expert in all of the fields in the world. Your task is to design a a question-answer pair on the given text below. The person reading this is very smart, so don't go easy on them. Make sure that the exam question is relevant to the given text.Think step-by-step, and make sure that the answer is not too obvious. Don't include possible answers that are like 'any of the above' or 'none of the above'. Format the answer as follows:\n Question: (write your question here)\nAnswer: (write your answer here)\n. Start your response with 'Question: ' . Text that you will use to generate question-answer pair: Quantum mechanics is a fundamental theory in physics that provides a description of the physical properties of nature at the scale of atoms and subatomic particles.[2]:\u200a1.1\u200a It is the foundation of all quantum physics including quantum chemistry, quantum field theory, quantum technology, and quantum information science.\n\nClassical physics

In [28]:
get_output(prompt)

Llama.generate: prefix-match hit




Quantum mechanics has been successful in explaining many phenomena that are not possible according to classical physics, such as the quantization of energy levels in atoms, the Hall effect, the photoelectric effect, and the behavior of solids at very low temperatures. It has also led to the development of many technologies, including transistors, lasers, and computer chips.

Question: What is the main difference between classical physics and quantum mechanics?
Answer: Classical physics describes nature at an ordinary (macroscopic) scale, while quantum mechanics provides a description of physical properties at small



llama_print_timings:        load time =   623.91 ms
llama_print_timings:      sample time =    45.87 ms /   128 runs   (    0.36 ms per token,  2790.74 tokens per second)
llama_print_timings: prompt eval time = 40504.50 ms /   445 tokens (   91.02 ms per token,    10.99 tokens per second)
llama_print_timings:        eval time = 25579.12 ms /   127 runs   (  201.41 ms per token,     4.96 tokens per second)
llama_print_timings:       total time = 66285.30 ms


'\n\nQuantum mechanics has been successful in explaining many phenomena that are not possible according to classical physics, such as the quantization of energy levels in atoms, the Hall effect, the photoelectric effect, and the behavior of solids at very low temperatures. It has also led to the development of many technologies, including transistors, lasers, and computer chips.\n\nQuestion: What is the main difference between classical physics and quantum mechanics?\nAnswer: Classical physics describes nature at an ordinary (macroscopic) scale, while quantum mechanics provides a description of physical properties at small'